# **Skenario 4 - CNN DenseNet121**

Skenario 4 menggunakan model Convolutional Neural Network (CNN) dengan arsitektur DenseNet121 untuk melakukan klasifikasi pada data citra otak.

---

## 1. Import packages

Import library yang dibutuhkan untuk pemrosesan data, image processing, modelling dan visualisasi

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import pickle

from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.backend import clear_session
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## 2. Data Preparation

### 2.1 Load dan Eksplorasi Data
cari dan cetak data `selected_image.npz` di dalam struktur direktori dan menampilkan informasi tentang jumlah slice untuk setiap label dan plane

In [2]:
def load_and_preprocess_data(img_size):
    base_dir = r"D:\Users\RESA\Coding\Alzheimer-Classification-with-CNN-SVM\Notebook\Preprocessing\image_selected.npz"
    
    # Load data from npz file
    loaded_data = np.load(base_dir, allow_pickle=True)
    loaded_combined_slices = loaded_data[list(loaded_data.keys())[0]]

    # Prepare X and Y lists
    X = []
    Y = []

    # Create a label mapping for your classes
    label_mapping = {'AD': 0, 'CN': 1, 'EMCI': 2, 'LMCI': 3}

    # Iterate through loaded data
    for label, plane_slices in loaded_combined_slices.item().items():
        for plane, slices in plane_slices.items():
            for selected_slice in slices:
                position, resized_slice = selected_slice

                # Resize the slice to the specified img_size
                img_arr = cv2.resize(resized_slice, (img_size, img_size))

                # Append data to X and Y
                X.append(img_arr)
                Y.append(label_mapping[label])

    # Convert lists to numpy arrays
    X = np.array(X)
    Y = np.array(Y)

    # Perform K-fold split
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    # Convert labels to categorical format
    Y = to_categorical(Y, num_classes=len(set(Y)))

    # Data augmentation using ImageDataGenerator
    datagen = ImageDataGenerator(
        rotation_range=360,
        horizontal_flip=True
    )

    return X, Y, kf, datagen

In [3]:
img_size = 224
epochs = 10
batch_size = 32
data_dictionary = {}

X, Y, kf, datagen = load_and_preprocess_data(img_size)

In [4]:
print("X shape:", X.shape)
print("Y shape:", Y.shape)

X shape: (7200, 224, 224)
Y shape: (7200, 4)


In [5]:
unique_labels, counts = np.unique(np.argmax(Y, axis=1), return_counts=True)
class_distribution = dict(zip(unique_labels, counts))

print("Distribusi Kelas setelah One-Hot Encoding:")
print(class_distribution)

Distribusi Kelas setelah One-Hot Encoding:
{0: 1800, 1: 1800, 2: 1800, 3: 1800}


In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [7]:
print("Dimensi X_train:", X_train.shape)
print("Dimensi Y_train:", Y_train.shape)
print("Dimensi X_test:", X_test.shape)
print("Dimensi Y_test:", Y_test.shape)

Dimensi X_train: (5760, 224, 224)
Dimensi Y_train: (5760, 4)
Dimensi X_test: (1440, 224, 224)
Dimensi Y_test: (1440, 4)


In [8]:
print("Jumlah sampel dalam X_train:", len(X_train))
print("Jumlah sampel dalam Y_train:", len(Y_train))
print("Jumlah sampel dalam X_test:", len(X_test))
print("Jumlah sampel dalam Y_test:", len(Y_test))

Jumlah sampel dalam X_train: 5760
Jumlah sampel dalam Y_train: 5760
Jumlah sampel dalam X_test: 1440
Jumlah sampel dalam Y_test: 1440


In [15]:
def create_densenet121(img_size, num_classes):
  base_model = DenseNet121(
    include_top=False,
    weights="imagenet",
    input_shape=(img_size, img_size, 3),
    pooling=None  # Remove the global average pooling here
  )

  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(1024, activation='relu')(x)
  predictions = Dense(num_classes, activation='softmax')(x)  # Specify the number of classes here

  densenet_model = Model(inputs=base_model.input, outputs=predictions)

  for layer in base_model.layers:
    layer.trainable = False

  densenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  return densenet_model

In [16]:
def cross_validate(create_model_fn, X, Y, kf, datagen, batch_size=32, epochs=10):
  entire_history = []
  best_history = None
  best_val_accuracy = -np.inf
  best_model = None
  best_val_x = None
  best_val_y = None

  for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]

    # Convert labels to categorical format
    num_classes = np.unique(np.argmax(Y, axis=1)).size
    Y_train = to_categorical(Y_train, num_classes=num_classes)
    Y_val = to_categorical(Y_val, num_classes=num_classes)

    clear_session()

    # Create a new instance of the model with reset weights
    model = create_model_fn(img_size=224, num_classes=num_classes)

    train_generator = datagen.flow(np.expand_dims(X_train, axis=-1), Y_train, batch_size=batch_size)

    model_history = model.fit(train_generator, steps_per_epoch=len(X_train) // batch_size, epochs=epochs)

    entire_history.append(model_history.history)
    current_val_accuracy = model_history.history['accuracy'][-1]

    if current_val_accuracy > best_val_accuracy:
      best_val_accuracy = current_val_accuracy
      best_history = model_history.history
      best_model = model
      best_val_x = X_val
      best_val_y = Y_val

    del model
    del model_history
    del X_train
    del X_val
    del Y_train
    del Y_val
    gc.collect()

  return best_history, best_model, best_val_x, best_val_y, entire_history